# train models

## 1. Train deep learning model

### 1.1 Load & process data

In [15]:
from deepLearning.dataloader import MacroData
from torch.utils.data import DataLoader
import torch
import pandas as pd
import utils


na_thresh = 500
df = pd.read_pickle("../../data/macro/consolidated_macro_data.pkl")
df = df.dropna(thresh=na_thresh, axis=1).dropna(axis=0)
df = df.T.drop_duplicates().T
processor = utils.PreProcessor(df)

md = MacroData(48, processor.train)
DEVICE = torch.device("cpu")
DTYPE = torch.float32
def collate_fn(data, device=DEVICE, dtype=DTYPE):
    return torch.tensor(data, dtype=dtype).to(device)
dataloader = DataLoader(md, batch_size=32, collate_fn=collate_fn)

/Users/paulp/Projects/CondMacroAssetAlloc/src/macroEncoding/utils/__init__.py:37: RuntimeWarning: divide by zero encountered in divide
  res[offset:] = np.log(input[offset:] / input[:-offset])
/Users/paulp/Projects/CondMacroAssetAlloc/src/macroEncoding/utils/__init__.py:37: RuntimeWarning: divide by zero encountered in log
  res[offset:] = np.log(input[offset:] / input[:-offset])
/Users/paulp/Projects/CondMacroAssetAlloc/src/macroEncoding/utils/__init__.py:37: RuntimeWarning: invalid value encountered in log
  res[offset:] = np.log(input[offset:] / input[:-offset])
/Users/paulp/Projects/CondMacroAssetAlloc/src/macroEncoding/utils/__init__.py:37: RuntimeWarning: divide by zero encountered in divide
  res[offset:] = np.log(input[offset:] / input[:-offset])
/Users/paulp/Projects/CondMacroAssetAlloc/src/macroEncoding/utils/__init__.py:37: RuntimeWarning: divide by zero encountered in log
  res[offset:] = np.log(input[offset:] / input[:-offset])
/Users/paulp/Projects/CondMacroAssetAlloc/src

In [16]:
X = next(iter(dataloader))

In [17]:
X.shape

torch.Size([32, 48, 60])

In [58]:
import torch.nn as nn
from typing import Tuple


class AutoEncoder(nn.Module):
    def __init__(self, 
                 window_size: int=60,
                 num_transformer_layers: int=3,
                 nhead: int=10,
                 dim: int=60,
                 encoding_dims: int=5):
        super().__init__()
        self.window_size = window_size
        self.dim = dim
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=nhead, batch_first=True)
        decoder_layer = nn.TransformerDecoderLayer(d_model=dim, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_transformer_layers)
        self.linear_encoder = nn.Sequential(
            nn.Flatten(1, -1),
            nn.Linear(dim * window_size, dim * window_size // 4),
            nn.Linear(dim * window_size // 4, dim * window_size // 4 ** 2),
            nn.Linear(dim * window_size // 4 ** 2, dim * window_size // 4 ** 3),
            nn.Linear(dim * window_size // 4 ** 3, encoding_dims)
        )
        self.linear_decoder = nn.Sequential(
            nn.Linear(encoding_dims, dim * window_size // 4 ** 3),
            nn.Linear(dim * window_size // 4 ** 3, dim * window_size // 4 ** 2),
            nn.Linear(dim * window_size // 4 ** 2, dim * window_size // 4),
            nn.Linear(dim * window_size // 4, dim * window_size),
            nn.Unflatten(-1, (self.window_size, self.dim)),
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=num_transformer_layers)
        

    def encode(self, x: torch.tensor) -> Tuple[torch.tensor]:
        x_ = self.transformer_encoder(x)
        # x_ = torch.flatten(x_, 1, 2)
        z = self.linear_encoder(x_)
        return x_, z
    
    def decode(self, z: torch.tensor, memory: torch.tensor) -> torch.tensor:
        y_ = self.linear_decoder(z)
        y = self.transformer_decoder(y_, memory=memory)
        return y

    def forward(self) -> torch.tensor:
        x_, z = self.encode(X)
        y_ = self.decode(z, x_)
        return y_
    
    def __call__(self, x) -> torch.tensor:
        return self.forward(x)


In [59]:
ae = AutoEncoder(window_size=48)

In [60]:
x_, z = ae.encode(X)
y_ = ae.decode(z, x_)

In [62]:
from torch.optim import AdamW=
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
import tensorx as tx


lr = 1e-3
num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(lr=lr, betas=(0.9, 0.99), eps=1e-4)


for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = ae.(inputs)

        # Compute loss
        loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Update parameters
        optimizer.step()

    # Log the parameters using TensorX
    tx.summary_parameters(autoencoder, epoch)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

tensor(-0.1275, grad_fn=<DivBackward1>)